In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
GROQ_API_KEY=os.getenv("GROQ_API_KEY")

In [5]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [6]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content='The translation of "I love programming" to French is:\n\n"J\'adore le programmation."', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 55, 'total_tokens': 77, 'completion_time': 0.029333333, 'prompt_time': 0.002776838, 'queue_time': 0.048086898, 'total_time': 0.032110171}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_a4265e44d5', 'finish_reason': 'stop', 'logprobs': None}, id='run-25cc8703-35a2-4bc3-9b3b-589893917c3b-0', usage_metadata={'input_tokens': 55, 'output_tokens': 22, 'total_tokens': 77})

In [7]:
print(ai_msg.content)

The translation of "I love programming" to French is:

"J'adore le programmation."


In [8]:
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.chains import create_history_aware_retriever
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import MessagesPlaceholder

In [9]:
model = ChatGroq(model="llama3-8b-8192")

In [10]:
model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content='Hi Bob! Nice to meet you! Is there something I can help you with or would you like to chat?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 15, 'total_tokens': 39, 'completion_time': 0.02, 'prompt_time': 0.005999195, 'queue_time': 1.124645742, 'total_time': 0.025999195}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None}, id='run-0697c255-cfe1-4818-9eda-5cfbf15b5f70-0', usage_metadata={'input_tokens': 15, 'output_tokens': 24, 'total_tokens': 39})

In [11]:
parser = StrOutputParser()

In [12]:
parser.invoke(model.invoke("hi"))

"Hi! It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [13]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\jeetu\Desktop\chatbot\chat_env\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jeetu\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
file_path="Data/Sample-NITI.pdf"


In [27]:
from langchain_community.document_loaders import PyPDFLoader,DirectoryLoader
import sys
sys.path.append('../..')

In [28]:
# Extract Data from the pdf file
def load_pdf_file(data):
    loader=DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents
    

In [32]:
extracted_data=load_pdf_file(data='C:\\Users\\jeetu\\Desktop\\chatbot\\Data\\')

In [34]:
#extracted_data

In [ ]:
#Split the Data into Text Chunks

def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [36]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks",len(text_chunks))

Length of Text Chunks 123


In [38]:
# text_chunks

In [39]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

In [40]:
def dowload_hugging_face_embedding():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [41]:
embeddings=dowload_hugging_face_embedding()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\jeetu\Desktop\chatbot\chat_env\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jeetu\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [42]:
query_result=embeddings.embed_query("Hello world")
print("Length",len(query_result))

Length 384


In [46]:
# query_result

In [47]:

vectorstore = Chroma.from_documents(documents=text_chunks, embedding=embeddings)


In [48]:
retriever = vectorstore.as_retriever()

In [49]:
retriever
     

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x0000024702AB2600>, search_kwargs={})

In [50]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer the question "
    "If you don't know the answer, say that you don't know."
    "Use three sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
)
     

In [51]:
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [52]:

question_answering_chain=create_stuff_documents_chain(model, chat_prompt)

In [53]:

rag_chain = create_retrieval_chain(retriever, question_answering_chain)

In [55]:
response = rag_chain.invoke({"input":"What role does energy modeling play in India's integrated energy planning and policy analysis?"})

In [56]:
retriever_prompt = (
    "Given a chat history and the latest user question which might reference context in the chat history,"
    "formulate a standalone question which can be understood without the chat history."
    "Do NOT answer the question, just reformulate it if needed and otherwise return it as is."
)

In [57]:
contextualize_q_prompt  = ChatPromptTemplate.from_messages(
    [
        ("system", retriever_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),


     ]
)

In [58]:
history_aware_retriever = create_history_aware_retriever(model,retriever,contextualize_q_prompt)

In [59]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [60]:
question_answer_chain = create_stuff_documents_chain(model, qa_prompt)
     

In [61]:
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [62]:
from langchain_core.messages import HumanMessage, AIMessage

In [63]:

chat_history = []

In [64]:
question1 = "How does India's Nationally Determined Contribution (NDC) target impact long-term electricity generation and emissions reduction?"

In [65]:
message1= rag_chain.invoke({"input": question1, "chat_history": chat_history})

In [66]:
message1["answer"]

"According to the provided context, India's NDC target aims to reduce the emissions intensity of its GDP by 33%-35% from 2005 levels by 2030, and to have 40% of cumulative electric power installed capacity from non-fossil sources. This target is expected to lead to significant improvement in emissions intensity, with a 41% reduction in the NDC scenario by 2032 compared to 2017 levels, and an 80% reduction compared to 2005 levels."

In [67]:
chat_history.extend(
    [
        HumanMessage(content=question1),
        AIMessage(content=message1["answer"]),
    ]
)

In [68]:
chat_history

[HumanMessage(content="How does India's Nationally Determined Contribution (NDC) target impact long-term electricity generation and emissions reduction?", additional_kwargs={}, response_metadata={}),
 AIMessage(content="According to the provided context, India's NDC target aims to reduce the emissions intensity of its GDP by 33%-35% from 2005 levels by 2030, and to have 40% of cumulative electric power installed capacity from non-fossil sources. This target is expected to lead to significant improvement in emissions intensity, with a 41% reduction in the NDC scenario by 2032 compared to 2017 levels, and an 80% reduction compared to 2005 levels.", additional_kwargs={}, response_metadata={})]

In [69]:
second_question = "Can I pay using PayPal?"
message2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(message2["answer"])

I don't know. This conversation is about India's Nationally Determined Contribution (NDC) target and its impact on long-term electricity generation and emissions reduction. I'm just an assistant, I don't have information about payment options or transactions.


In [70]:
store = {}

In [71]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [72]:
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [73]:
conversational_rag_chain.invoke(
    {"input": "How does India's Nationally Determined Contribution (NDC) target impact long-term electricity generation and emissions reduction?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

"India's Nationally Determined Contribution (NDC) target aims to reduce emissions intensity of its GDP by 33%-35% from 2005 level, increase non-fossil installed capacity to 40% by 2030, and create an additional carbon sink."

In [74]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content="How does India's Nationally Determined Contribution (NDC) target impact long-term electricity generation and emissions reduction?", additional_kwargs={}, response_metadata={}), AIMessage(content="India's Nationally Determined Contribution (NDC) target aims to reduce emissions intensity of its GDP by 33%-35% from 2005 level, increase non-fossil installed capacity to 40% by 2030, and create an additional carbon sink.", additional_kwargs={}, response_metadata={})])}

In [76]:
conversational_rag_chain.invoke(
    {"input": "What are the key challenges India faces in achieving a balance between energy access, security, and sustainability?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'India faces the trilemma of achieving objectives of higher energy access alongside higher energy security and higher sustainability, where energy security and sustainability are mutually reinforcing, but the goal of accessibility will conflict with the goal of sustainability.'

In [77]:
for message in store["abc123"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

User: How does India's Nationally Determined Contribution (NDC) target impact long-term electricity generation and emissions reduction?

AI: India's Nationally Determined Contribution (NDC) target aims to reduce emissions intensity of its GDP by 33%-35% from 2005 level, increase non-fossil installed capacity to 40% by 2030, and create an additional carbon sink.

User: Can I pay using PayPal?

AI: I don't have information about payment options.

User: What are the key challenges India faces in achieving a balance between energy access, security, and sustainability?

AI: India faces the trilemma of achieving objectives of higher energy access alongside higher energy security and higher sustainability, where energy security and sustainability are mutually reinforcing, but the goal of accessibility will conflict with the goal of sustainability.



In [78]:
conversational_rag_chain.invoke(
    {"input": "Do you support cryptocurrency payments?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

"I don't know."

In [79]:
store

{'abc123': InMemoryChatMessageHistory(messages=[HumanMessage(content="How does India's Nationally Determined Contribution (NDC) target impact long-term electricity generation and emissions reduction?", additional_kwargs={}, response_metadata={}), AIMessage(content="India's Nationally Determined Contribution (NDC) target aims to reduce emissions intensity of its GDP by 33%-35% from 2005 level, increase non-fossil installed capacity to 40% by 2030, and create an additional carbon sink.", additional_kwargs={}, response_metadata={}), HumanMessage(content='Can I pay using PayPal?', additional_kwargs={}, response_metadata={}), AIMessage(content="I don't have information about payment options.", additional_kwargs={}, response_metadata={}), HumanMessage(content='What are the key challenges India faces in achieving a balance between energy access, security, and sustainability?', additional_kwargs={}, response_metadata={}), AIMessage(content='India faces the trilemma of achieving objectives of h

In [80]:
for message in store["abc123"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

User: How does India's Nationally Determined Contribution (NDC) target impact long-term electricity generation and emissions reduction?

AI: India's Nationally Determined Contribution (NDC) target aims to reduce emissions intensity of its GDP by 33%-35% from 2005 level, increase non-fossil installed capacity to 40% by 2030, and create an additional carbon sink.

User: Can I pay using PayPal?

AI: I don't have information about payment options.

User: What are the key challenges India faces in achieving a balance between energy access, security, and sustainability?

AI: India faces the trilemma of achieving objectives of higher energy access alongside higher energy security and higher sustainability, where energy security and sustainability are mutually reinforcing, but the goal of accessibility will conflict with the goal of sustainability.

User: Do you support cryptocurrency payments?

AI: I don't know.

